In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import time
import functions

In [ ]:
filepath ='data/'
csvfilenames = ['confinement_training.csv', 'member_information.csv', 'labresults_training.csv', 'medical_training.csv', 'rx_training.csv']
txtfilenames = ['lu_diagnosis.txt', 'lu_procedure-fake.txt', 'lu_ndc.txt' ]

with open(filepath+csvfilenames[0]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

confinement = pd.DataFrame(data)
confinement.columns = confinement.iloc[0]
confinement = confinement[1:] #sets first row as header

with open(filepath+csvfilenames[1]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open(filepath+csvfilenames[3]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

allmedical = pd.DataFrame(data)
allmedical.columns = allmedical.iloc[0]
allmedical = allmedical[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

In [ ]:
allmedical['STD_COST']=allmedical['STD_COST'].apply(pd.to_numeric) #change entries to numbers
rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers

In [ ]:
allmedical['wanted']=True

medical_wanted = allmedical[('630' >=  allmedical.DIAG1) | (allmedical.DIAG1 >= '67999')]
medical_wanted = medical_wanted[('630' >=  medical_wanted.DIAG2) | (medical_wanted.DIAG2 >= '67999')]

medical_wanted = medical_wanted[('800' >=  medical_wanted.DIAG1) | (medical_wanted.DIAG1 >= '95999')]
medical_wanted = medical_wanted[('800' >=  medical_wanted.DIAG2) | (medical_wanted.DIAG2 >= '95999')]

medical_wanted = medical_wanted[medical_wanted['DIAG1'].str.startswith('E') == False]
medical_wanted = medical_wanted[medical_wanted['DIAG2'].str.startswith('E') == False]

medical_wanted = medical_wanted[(medical_wanted['DIAG1'].str.startswith('V3') == False) & (medical_wanted['DIAG1'].str.startswith('V22') == False) & (medical_wanted['DIAG1'].str.startswith('V23') == False) & (medical_wanted['DIAG1'].str.startswith('V24') == False) & (medical_wanted['DIAG1'].str.startswith('V28') == False) & (medical_wanted['DIAG1'].str.startswith('V91') == False)]
medical_wanted = medical_wanted[(medical_wanted['DIAG2'].str.startswith('V3') == False) & (medical_wanted['DIAG2'].str.startswith('V22') == False) & (medical_wanted['DIAG2'].str.startswith('V23') == False) & (medical_wanted['DIAG2'].str.startswith('V24') == False) & (medical_wanted['DIAG2'].str.startswith('V28') == False) & (medical_wanted['DIAG2'].str.startswith('V91') == False)]


In [ ]:
#This has been improved upon above. Run that instead

tick = time.clock()
k=0
while k < len(allmedical):
    if not (functions.wanted(allmedical.iat[k,9]) and functions.wanted(allmedical.iat[k,10])):
        allmedical.iat[k, 20] = False
    k += 1
    if k % 100000 == 0:
        print k
        tock = time.clock()
        print tock-tick

In [ ]:
wantedmedical = allmedical[allmedical.wanted]

In [ ]:
#This has been improved upon below. Do not run this one.

tick = time.clock()
members['totalcost']=0
#add rxdata to member's total cost
k=0
while k < len(rxdata):
    members.at[rxdata.iloc[k]['PATID'],'totalcost']+=rxdata.iloc[k]['STD_COST']
    k += 1
    if k % 100000 == 0:
        print k
        tock=time.clock()
        print tock-tick
print 'Done with rxdata! Moving on to medical data'
tick=time.clock()
k=0
while k < len(wantedmedical):
    members.at[wantedmedical.iloc[k]['PATID'],'totalcost']+=wantedmedical.iloc[k]['STD_COST']
    k += 1
    if k % 100000 == 0:
        print k
        tock=time.clock()
        print tock-tick
print "yaaaaaaaay"

In [ ]:
members['totalcost'] = 0.0 #add rxdata to member's total cost
members['rxcost'] = 0.0
members['medicalcost'] = 0.0

dummy = rxdata.groupby('PATID')['STD_COST'].sum()
#print dummy

for k in range(len(dummy)):
    members.at[dummy.index[k],'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'rxcost'] = dummy.iat[k]
    #print type(members.at[dummy.index[k],'totalcost'])
    
dummy = medical_wanted.groupby('PATID')['STD_COST'].sum()
#print dummy.head()

for k in range(len(dummy)):
    members.at[dummy.index[k], 'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'medicalcost'] = dummy.iat[k]
    
#print members.head()


In [ ]:
ser = allmedical['STD_COST']
ser.plot.density()
plt.xlim([-5000,5000])
plt.show()

In [ ]:
allmedical.sort_values(["STD_COST"], ascending = [1])

In [ ]:
location_series = medical_wanted.groupby('POS')['STD_COST'].mean()

location_series.plot.bar()
plt.ylim([0,2000])
plt.show()

In [ ]:
# Add new column for region (should find a way to avoid this with groupby, but can't currently)
members['region'] = members.apply(lambda row: functions.US_region(row), axis=1)

region_series = members.groupby('region')['totalcost'].mean()

region_series.plot.bar()
plt.show()

In [ ]:
reload(functions)